# Cas13 Sequence to Sequence Optimal Guide Design

In [4]:
pip install biopython

     |████████████████████████████████| 2.2 MB 6.7 MB/s eta 0:00:01
You should consider upgrading via the '/Users/priyankashrestha/rnn/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
Entrez.email = "shrestp@stanford.edu"
import pandas as pd
import re
from pprint import pprint
import matplotlib.pyplot as plt

In [3]:
corona_guides_path = r"/Users/priyankashrestha/Documents/research/seq2seq/Coronavirus.MN908947_NY1-PV08001.S_CasRxguides.csv"
corona_genome_path = r"/Users/priyankashrestha/Documents/research/seq2seq/Coronavirus.MN908947_NY1-PV08001.fasta"

In [4]:
df = pd.read_csv(corona_guides_path, usecols=['GuideSeq', 'GuideScores'])
corona_guides = df['GuideSeq'].values.tolist()
corona_genome = SeqIO.parse(corona_genome_path, "fasta")

In [5]:
df

,GuideSeq,GuideScores
0,CCACATAATAAGCTGCAGCACCA,1.659528
1,CACATAATAAGCTGCAGCACCAG,1.578299
2,CCCACATAATAAGCTGCAGCACC,1.550406
3,AACCCACATAATAAGCTGCAGCA,1.547846
4,ACATAATAAGCTGCAGCACCAGC,1.544357
...,...,...
3602,TTTTAAAATAACCATCAATATTC,-0.174768
3603,AAACATTAGAACCTGTAGAATAA,-0.185012
3604,ATTTTAAAATAACCATCAATATT,-0.191938
3605,TATTTTAAAATAACCATCAATAT,-0.196843


In [6]:
# make a dictionary mapping each guide sequence to its corresponding score
guide_to_score = pd.Series(df.GuideScores.values,index=df.GuideSeq).to_dict()

In [7]:
genome = [] # list with a single entry containing the corona virus genome as Seq objects
guides = [] # list containing all of the coronavirus guides as Seq objects

for seq_record in corona_genome:
    genome = seq_record.seq
    
for guide in corona_guides:
    seq_guide = Seq(guide)
    guides.append(seq_guide)
    

In [8]:
print("The total number of guides in the dataset is:", len(guides))
print("The Coronavirus genome is " + str(len(str(genome))) + " nucleotides long")

The total number of guides in the dataset is: 3607
The Coronavirus genome is 29903 nucleotides long


step 1: split genome into 100 nucleotide sections

step 2: find all guides that are in that region

step 3: find the highest scoring guide in that region 

In [9]:
def genome_splitter(s, n):
    """ produce 's'- nucleotide chunks from genome. """
    for start in range(0, len(genome)):
        yield s[start:start+n]
        
def rc(guide):
    """ return the reverse complement of a guide """
    return str(Seq(guide).reverse_complement())

def guide_in_segment(guide, segment):
    """ return whether or not the guide is in the segment """
    return guide in segment or rc(guide) in segment

def find_guides(split_genome, guide_list):
    """ find all of the guides that fall within each section of the split genome"""
    dat = {segment : [] for segment in split_genome}
    for guide in guide_list:
        for segment in split_genome:
            if (guide_in_segment(guide, segment)):
                dat[segment].append(guide)
    return dat

def find_best_guide(segment_to_guides, guide_to_score):
    new_dat = {}
    guide_scores = []
    for segment, guide_list in segment_to_guides.items():
        for guide in guide_list:
            guide_score = [guide_to_score[guide], guide]
            guide_scores.append(guide_score)
            max_score, best_guide = max(guide_scores)
            new_dat[segment] = best_guide
    return new_dat  

def prepare_inputs(guide_list):
    # append the start character (\t) and the end character (\n) to the input sequences
    prepared_guides = []
    for guide in guide_list:
        prepared_guides.append('\t' + str(guide_list[i]) + '\n')
    return prepared_guides    

In [10]:
split_genome = []
for subset in genome_splitter(genome, 100):
    split_genome.append(str(subset))

In [14]:
segment_to_guides = find_guides(split_genome, corona_guides)

In [15]:
segment_to_best_guide = find_best_guide(segment_to_guides, guide_to_score)

## Data Preparation for Model

In [17]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

In [18]:
# vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [19]:
# initialize data with lists from above
input_texts = list(segment_to_best_guide.keys())
target_texts = list(segment_to_best_guide.values())

In [20]:
# types of characters in input and output = 'A', 'C', 'G', 'T'

for input_text in input_texts:
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
            
for target_text in target_texts:
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [21]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [22]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 3877
Number of unique input tokens: 4
Number of unique output tokens: 4
Max sequence length for inputs: 100
Max sequence length for outputs: 23


In [23]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [24]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [25]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    #encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    #decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    #decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [26]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens), name='input')
encoder = keras.layers.LSTM(latent_dim, return_state=True, name = 'lstm_1')
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens), name='input_2')

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True, name='lstm_2')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax", name='dense_1')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, 4)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 4)]    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 267264      input[0][0]                      
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  267264      input_2[0][0]                    
                                                                 lstm_1[0][1]                 

In [28]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

model.save("predict_guide_test")

Epoch 1/100
49/49 [==============================] - 12s 192ms/step - loss: 1.2046 - accuracy: 0.4198 - val_loss: 0.8912 - val_accuracy: 0.6076
Epoch 2/100
49/49 [==============================] - 9s 175ms/step - loss: 0.9483 - accuracy: 0.5552 - val_loss: 0.7677 - val_accuracy: 0.7826
Epoch 3/100
49/49 [==============================] - 9s 179ms/step - loss: 0.7926 - accuracy: 0.6985 - val_loss: 0.3914 - val_accuracy: 1.0000
Epoch 4/100
49/49 [==============================] - 9s 184ms/step - loss: 0.5853 - accuracy: 0.7877 - val_loss: 0.4276 - val_accuracy: 0.8696
Epoch 5/100
49/49 [==============================] - 9s 187ms/step - loss: 0.4263 - accuracy: 0.8399 - val_loss: 0.1497 - val_accuracy: 0.9565
Epoch 6/100
49/49 [==============================] - 9s 188ms/step - loss: 0.2759 - accuracy: 0.9042 - val_loss: 0.1857 - val_accuracy: 0.9300
Epoch 7/100
49/49 [==============================] - 9s 179ms/step - loss: 0.2325 - accuracy: 0.9221 - val_loss: 0.0375 - val_accuracy: 1.000

Epoch 58/100
49/49 [==============================] - 9s 177ms/step - loss: 0.0178 - accuracy: 0.9540 - val_loss: 0.0037 - val_accuracy: 0.9565
Epoch 59/100
49/49 [==============================] - 9s 177ms/step - loss: 0.0182 - accuracy: 0.9540 - val_loss: 0.0036 - val_accuracy: 0.9565
Epoch 60/100
49/49 [==============================] - 9s 178ms/step - loss: 0.0181 - accuracy: 0.9542 - val_loss: 0.0034 - val_accuracy: 0.9565
Epoch 61/100
49/49 [==============================] - 9s 176ms/step - loss: 0.0171 - accuracy: 0.9546 - val_loss: 0.0034 - val_accuracy: 0.9565
Epoch 62/100
49/49 [==============================] - 9s 192ms/step - loss: 0.0169 - accuracy: 0.9546 - val_loss: 0.0036 - val_accuracy: 0.9565
Epoch 63/100
49/49 [==============================] - 9s 181ms/step - loss: 0.0157 - accuracy: 0.9550 - val_loss: 0.0034 - val_accuracy: 0.9565
Epoch 64/100
49/49 [==============================] - 8s 174ms/step - loss: 0.0159 - accuracy: 0.9548 - val_loss: 0.0033 - val_accuracy:

INFO:tensorflow:Assets written to: predict_guide_test/assets


INFO:tensorflow:Assets written to: predict_guide_test/assets


In [29]:
model = keras.models.load_model("predict_guide_test")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)

In [30]:
print(model.input)

[<KerasTensor: shape=(None, None, 4) dtype=float32 (created by layer 'input')>, <KerasTensor: shape=(None, None, 4) dtype=float32 (created by layer 'input_2')>]


In [31]:
decoder_inputs

<KerasTensor: shape=(None, None, 4) dtype=float32 (created by layer 'input_2')>

In [32]:
decoder_states_inputs

[<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_3')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_4')>]

In [33]:
decoder_outputs

<KerasTensor: shape=(None, None, 4) dtype=float32 (created by layer 'dense_1')>

In [34]:
decoder_states

[<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm_2')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm_2')>]

In [35]:
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [36]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [37]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    #target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [38]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input DNA:", input_texts[seq_index])
    print("Predicted Optimal sgRNA:", decoded_sentence)
    print("Actual Optimal sgRNA:", target_texts[seq_index] )

-
Input DNA: TAAAGGTAGACTTATAATTAGAGAAAACAACAGAGTTGTTATTTCTAGTGATGTTCTTGTTAACAACTAAACGAACAATGTTTGTTTTTCTTGTTTTATT
Predicted Optimal sgRNA: ATAAAACAAGAAAAACAAAAGGGG
Actual Optimal sgRNA: AATAAAACAAGAAAAACAAACAT
-
Input DNA: AAAGGTAGACTTATAATTAGAGAAAACAACAGAGTTGTTATTTCTAGTGATGTTCTTGTTAACAACTAAACGAACAATGTTTGTTTTTCTTGTTTTATTG
Predicted Optimal sgRNA: ACATAATAGCATGGAACCAAGGGG
Actual Optimal sgRNA: AATAAAACAAGAAAAACAAACAT
-
Input DNA: AAGGTAGACTTATAATTAGAGAAAACAACAGAGTTGTTATTTCTAGTGATGTTCTTGTTAACAACTAAACGAACAATGTTTGTTTTTCTTGTTTTATTGC
Predicted Optimal sgRNA: CAAGATAACAAGAAAAACAAGGGG
Actual Optimal sgRNA: GCAATAAAACAAGAAAAACAAAC
-
Input DNA: AGGTAGACTTATAATTAGAGAAAACAACAGAGTTGTTATTTCTAGTGATGTTCTTGTTAACAACTAAACGAACAATGTTTGTTTTTCTTGTTTTATTGCC
Predicted Optimal sgRNA: GTAAGATTAGCATACAGACTAGGG
Actual Optimal sgRNA: GGCAATAAAACAAGAAAAACAAA
-
Input DNA: GGTAGACTTATAATTAGAGAAAACAACAGAGTTGTTATTTCTAGTGATGTTCTTGTTAACAACTAAACGAACAATGTTTGTTTTTCTTGTTTTATTGCCA
Predicted Optimal sgRNA: GTAAGATAACAATAAAACAAG

In [39]:
segment_to_best_guide['AGAGAAAACAACAGAGTTGTTATTTCTAGTGATGTTCTTGTTAACAACTAAACGAACAATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAG']

'CTGACTAGAGACTAGTGGCAATA'

#### open coronavirus guides file:
corona_genome = SeqIO.parse(sarscovgenome_path, "fasta")

genome = []
for seq_record in sarscov_genome:
    genome = str(seq_record.seq)

genome = genome[:-1]

In [7]:
#bring in fasta file of possiible grna and put them in a list
def make_list(grna_file):
    grna_list = []

    #create a list with all possible gRNA in order from most to least effective
    for elem in grna_file:
        grna_list.append(str(elem.seq))

    return grna_list

In [8]:
corona_guides = make_list(corona_guides)

In [9]:
corona_guides = corona_guides[:int(0.5*len(corona_guides))]

In [10]:
for i in range(len(sarscov_guides)):
    sarscov_guides[i] = '\t' + sarscov_guides[i] + '\n'

In [186]:
def format_genome_index(locations):
    for i in range (len(locations)):
        locations[i] = locations[i][locations[i].find(':') + 1:]
        locations[i] = re.split('-', locations[i])
    
    locations = locations[:int(0.5*len(locations))]
    return locations

def genome_parser(genome, index_list):
    dna = []
    for index in index_list:
        dna.append(genome[int(index[0]):int(index[1])])
    return dna

In [187]:
corona_guides_location = format_genome_index(corona_guides_location)

In [188]:
corona_dna = genome_parser(genome, corona_guides_location)

In [189]:
df_prepared = pd.DataFrame(list(zip(corona_dna, corona_guides)))

In [190]:
df_prepared

,0,1
0,ACTCATGCGTGAGCTTAACGGA,\tCCACATAATAAGCTGCAGCACCA\n
1,AACTCATGCGTGAGCTTAACGG,\tCACATAATAAGCTGCAGCACCAG\n
2,CTCATGCGTGAGCTTAACGGAG,\tCCCACATAATAAGCTGCAGCACC\n
3,CATGCGTGAGCTTAACGGAGGG,\tAACCCACATAATAAGCTGCAGCA\n
4,GAACTCATGCGTGAGCTTAACG,\tACATAATAAGCTGCAGCACCAGC\n
...,...,...
1798,GATCATCAGCACATCTAGGTTT,\tATCAAACCTCTTAGTACCATTGG\n
1799,GCTGTTGTTAAAATTTATTGTC,\tGACTTCCTAAACAATCTATACAG\n
1800,CATGTGGGCGAAATACCAGTGG,\tGAATATATTTTAAAATAACCATC\n
1801,AACAGAAATCAATACTGAGTCC,\tGACTTCTGTGCAGTTAACATCCT\n
